### 데이터 파일 생성

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%autosave 0
import warnings
warnings.filterwarnings(action='ignore')

import os
import time

import numpy as np
import pandas as pd
import seaborn as sb

from tensorflow.keras.models import Sequential  # class
from tensorflow.keras.models import load_model  # model 사용
from tensorflow.keras.layers import Dense       # 전결합
from tensorflow.keras.layers import Dropout     # 특정 node를 사용안함.
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten

from tensorflow.keras.callbacks import EarlyStopping   # 학습 자동 중지
from tensorflow.keras.callbacks import ModelCheckpoint # 우수한 학습 모델 파일 저장
from tensorflow.keras import regularizers 
from tensorflow.keras.utils import to_categorical   # one-hot 엔코딩

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.datasets import mnist

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split # 학습셋과 테스트셋의 분리 지원
from sklearn.model_selection import StratifiedKFold  # K겹 교차 검증

import matplotlib.pyplot as plt
from matplotlib import font_manager, rc

import platform 

if (platform.system() == 'Windows'):  # Windows, Linux, Darwin
    rc('font', family=font_manager.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name())
    path = '.' # Local
else:    
    plt.rc('font', family='NanumBarunGothic')  # Ubuntu 18.04 기준 한글 처리
    path = '/content/drive/My Drive/kd_ml/cnn/shape_ext' # Colab

os.chdir(path) # 기본 경로 설정    
    
plt.rcParams["font.size"] = 12         # 글자 크기
# plt.rcParams["figure.figsize"] = (10, 4) # 10:4의 그래프 비율
plt.rcParams['axes.unicode_minus'] = False  # minus 부호는 unicode 적용시 한글이 깨짐으로 설정

%matplotlib inline  

In [ ]:
import numpy as np

# 랜덤시드 고정시키기
np.random.seed(0)

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

# 데이터셋 생성하기
train_datagen = ImageDataGenerator(rotation_range=15,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.5,
                                   zoom_range=0.3,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

In [ ]:
img = load_img('./src/circle_k.jpg')
X = img_to_array(img)
print(X.shape) # 24 x 24 칼라(3) 이미지
print(type(X))
print(X)

(24, 24, 3)
<class 'numpy.ndarray'>
[[[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 ...

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[254. 254. 254.]
  [255. 255. 255.]
  [255. 255. 255.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]]


In [ ]:
# train_datagen.flow() 함수는 4차원 형태로 데이터를 받음
print(X.shape) # 24 x 24 칼라(3) 이미지
x = X.reshape((1,) + X.shape) # (1, ) + (24, 24, 3) # 차원 확장
print(x.shape) # Rank 4, 4차원 배열

(24, 24, 3)
(1, 24, 24, 3)


In [ ]:
# 훈련 폴더, 검증 폴더 자동 생성
import os

newfolder = './train'
if os.path.exists(newfolder) == False:
    os.mkdir(newfolder)

newfolder = './train/circle'
if os.path.exists(newfolder) == False:
    os.mkdir(newfolder)

newfolder = './train/rectangle'
if os.path.exists(newfolder) == False:
    os.mkdir(newfolder)

newfolder = './train/triangle'
if os.path.exists(newfolder) == False:
    os.mkdir(newfolder)


newfolder = './validation'
if os.path.exists(newfolder) == False:
    os.mkdir(newfolder)

newfolder = './validation/circle'
if os.path.exists(newfolder) == False:
    os.mkdir(newfolder)

newfolder = './validation/rectangle'
if os.path.exists(newfolder) == False:
    os.mkdir(newfolder)

newfolder = './validation/triangle'
if os.path.exists(newfolder) == False:
    os.mkdir(newfolder)
    

### 훈련 및 검증 이미지는 1개 더 생성할것
- 불규칙하게 생성시 1개가 적개 생성되는 문제 있음.
- 검은색 계열 원, 사각형, 삼각형 훈련용 각각 80개 생성

In [ ]:
img = load_img('./src/circle_k.jpg')
X = img_to_array(img)
x = X.reshape((1,) + X.shape) # (1, ) + (24, 24, 3)
i = 0
# 자동 중지가 안됨으로 break로 종료
for batch in train_datagen.flow(x, batch_size=1, 
                                save_to_dir='./train/circle',
                                save_prefix='k',
                                save_format='jpg'):
    i = i + 1
    if i >= 81:
        break;

In [ ]:
img = load_img('./src/rectangle_k.jpg')
X = img_to_array(img)
x = X.reshape((1,) + X.shape) # (1, ) + (24, 24, 3)
i = 0
# 자동 중지가 안됨으로 break로 종료
for batch in train_datagen.flow(x, batch_size=1, 
                                save_to_dir='./train/rectangle',
                                save_prefix='k',
                                save_format='jpg'):
    i = i + 1
    if i >= 81:
        break;

In [ ]:
img = load_img('./src/triangle_k.jpg')
X = img_to_array(img)
x = X.reshape((1,) + X.shape) # (1, ) + (24, 24, 3)
i = 0
# 자동 중지가 안됨으로 break로 종료
for batch in train_datagen.flow(x, batch_size=1, 
                                save_to_dir='./train/triangle',
                                save_prefix='k',
                                save_format='jpg'):
    i = i + 1
    if i >= 81:
        break;

#### 검은색 계열 원, 사각형, 삼각형 검증용 각각 20개 생성

In [ ]:
img = load_img('./src/circle_k.jpg')
X = img_to_array(img)
x = X.reshape((1,) + X.shape) # (1, ) + (24, 24, 3)
i = 0
# 자동 중지가 안됨으로 break로 종료
for batch in train_datagen.flow(x, batch_size=1, 
                                save_to_dir='./validation/circle',
                                save_prefix='k',
                                save_format='jpg'):
    i = i + 1
    if i >= 20:
        break;

In [ ]:
img = load_img('./src/rectangle_k.jpg')
X = img_to_array(img)
x = X.reshape((1,) + X.shape) # (1, ) + (24, 24, 3)
i = 0
# 자동 중지가 안됨으로 break로 종료
for batch in train_datagen.flow(x, batch_size=1, 
                                save_to_dir='./validation/rectangle',
                                save_prefix='k',
                                save_format='jpg'):
    i = i + 1
    if i >= 20:
        break;

In [ ]:
img = load_img('./src/triangle_k.jpg')
X = img_to_array(img)
x = X.reshape((1,) + X.shape) # (1, ) + (24, 24, 3)
i = 0
# 자동 중지가 안됨으로 break로 종료
for batch in train_datagen.flow(x, batch_size=1, 
                                save_to_dir='./validation/triangle',
                                save_prefix='k',
                                save_format='jpg'):
    i = i + 1
    if i >= 20:
        break;

#### 빨간색 계열 원, 사각형, 삼각형 훈련용 각각 80개 생성

In [ ]:
folders = ['circle', 'rectangle', 'triangle']  # 폴더명을 list에 저장
for folder in folders:
    print(folder)

circle
rectangle
triangle


In [ ]:
folders = ['circle', 'rectangle', 'triangle']
for folder in folders:
    img = load_img('./src/' + folder + '_r.jpg')
    X = img_to_array(img)
    x = X.reshape((1,) + X.shape) # (1, ) + (24, 24, 3)
    i = 0
    # 자동 중지가 안됨으로 break로 종료
    for batch in train_datagen.flow(x, batch_size=1, 
                                    save_to_dir='./train/' + folder,
                                    save_prefix='r',
                                    save_format='jpg'):
        i = i + 1
        if i >= 81:
            break;

#### 빨간색 계열 원, 사각형, 삼각형 검증용 각각 20개 생성

In [ ]:
folders = ['circle', 'rectangle', 'triangle']
for folder in folders:
    img = load_img('./src/' + folder + '_r.jpg')
    X = img_to_array(img)
    x = X.reshape((1,) + X.shape) # (1, ) + (24, 24, 3)
    i = 0
    # 자동 중지가 안됨으로 break로 종료
    for batch in train_datagen.flow(x, batch_size=1, 
                                    save_to_dir='./validation/' + folder,
                                    save_prefix='r',
                                    save_format='jpg'):
        i = i + 1
        if i >= 20:
            break;

#### 파란색 계열 원, 사각형, 삼각형 훈련용 각각 80개 생성

In [ ]:
folders = ['circle', 'rectangle', 'triangle']
for folder in folders:
    img = load_img('./src/' + folder + '_b.jpg')
    X = img_to_array(img)
    x = X.reshape((1,) + X.shape) # (1, ) + (24, 24, 3)
    i = 0
    # 자동 중지가 안됨으로 break로 종료
    for batch in train_datagen.flow(x, batch_size=1, 
                                    save_to_dir='./train/' + folder,
                                    save_prefix='b',
                                    save_format='jpg'):
        i = i + 1
        if i >= 81:
            break;

#### 파란색 계열 원, 사각형, 삼각형 검증용 각각 20개 생성

In [ ]:
folders = ['circle', 'rectangle', 'triangle']
for folder in folders:
    img = load_img('./src/' + folder + '_b.jpg')
    X = img_to_array(img)
    x = X.reshape((1,) + X.shape) # (1, ) + (24, 24, 3)
    i = 0
    # 자동 중지가 안됨으로 break로 종료
    for batch in train_datagen.flow(x, batch_size=1, 
                                    save_to_dir='./validation/' + folder,
                                    save_prefix='b',
                                    save_format='jpg'):
        i = i + 1
        if i >= 20:
            break;